In [1]:
# Arranco montando Drive para acceder a los archivos necesarios

root_path = 'drive/My Drive/Big Data/'
my_path = root_path + 'TP2/'

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Instalo Spark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from pyspark import SparkContext
sc = SparkContext("local", "MyProgram")

In [4]:
from pyspark.sql import SQLContext, Row, Window, functions as F
sqlContext = SQLContext(sc)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
# Leo input y divido en columnas
trips = sc.textFile(my_path + '/Trips/input/')                  
trips = trips.map(lambda t: t.split('\t'))   
trips = trips.map(lambda t: Row(id = int(t[0]), avenue = t[1], street = t[2], timestamp = int(t[3]), destination = t[4]))
tripsDataFrame = sqlContext.createDataFrame(trips)  

In [6]:
windowByID = Window.partitionBy('id').orderBy(F.asc('timestamp'))
route = tripsDataFrame.withColumn('previous_avenue', F.lag('avenue', 1).over(windowByID))
route = route.withColumn('previous_street', F.lag('street', 1).over(windowByID))

def searchMax(route, position, position2):
  windowByPosition = Window.partitionBy(position)
  positionDataFrame = route.where(f"{position} = previous_{position} and {position2} != previous_{position2}")
  positionDataFrame = positionDataFrame.withColumn('blocks', F.count('id').over(windowByPosition))
  positionDataFrame = positionDataFrame.select(position, 'blocks')
  positionMax = positionDataFrame.sort(positionDataFrame.blocks.desc()).first()
  return positionMax

avenueMax = searchMax(route, 'avenue', 'street')
streetMax = searchMax(route, 'street', 'avenue')


In [10]:
print('Avenida más recorrida:', avenueMax[0], "se recorrieron", avenueMax[1], "cuadras")
print('Calle más recorrida:', streetMax[0], "se recorrieron", streetMax[1], "cuadras")

Avenida más recorrida: 94 se recorrieron 1679 cuadras
Calle más recorrida: 82 se recorrieron 1756 cuadras
